In [1]:
import numpy as np
import pandas as pd
import json
import http.client, urllib.parse

In [2]:
import voila

In [3]:
# voila --template=debug --VoilaExporter.template_file=debug.tpl

In [4]:
from datetime import date

today = date.today()
print("Today's date:", today)

Today's date: 2022-11-05


In [5]:
# !jupyter serverextension enable voila --sys-prefix

In [6]:
# for x in dict_data['data']:
#     df = pd.DataFrame.from_dict(x, orient='index',
#                        columns=['author', 'title', 'description', 'url','source','image','category','language','country','published_at'])

In [7]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output

In [8]:
# Image Widget

file = open("download.png", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='jpg',
                    width='300'
                )

label_headline = widgets.Label(
                    value='Image by Iconka',
                    style={'description_width': 'initial'}
                )

vbox_headline = widgets.VBox([image_headline, label_headline])

In [9]:
vbox_headline

In [10]:
# news source

source = widgets.Dropdown(
    options=[('General',''),('BBC', 'bbc'), ('CNN', 'cnn'), ('ESPN', 'espn')],
    value='',
    description='Source:',
)

In [11]:
# category

category = widgets.Dropdown(
    options=[('General', 'general'), ('Business', 'business'),('Entertainment','entertainment'),('Health', 'health'),('Science','science'),('Technology','technology')],
    value='general',
    description='Category:',
)

In [12]:
fill = widgets.Text(
    value='',
    placeholder='Enter Keyword(s)',
    description='Keyword:',
    disabled=False
)

In [13]:
#Date picker

date_start = widgets.DatePicker(value = today, description='Pick a Start')
date_end = widgets.DatePicker(value = today,description='Pick an End')

In [14]:
button_send = widgets.Button(
                description='Confirm',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        print("Loading...")
        print("Fetching" ,category.value, "news from", source.value, "with",fill.value, "as a keyword from", date_start.value,"to",date_end.value)

button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

In [15]:
vbox_text = widgets.VBox([source, category, fill, date_start, date_end, vbox_result])

In [16]:
page = widgets.HBox([vbox_headline, vbox_text])
display(page)

In [17]:
temp_start = date_start.value.strftime('%Y-%m-%d')
temp_end = date_end.value.strftime('%Y-%m-%d')
parsed_date = temp_start +','+temp_end

In [18]:
parsed_date

'2022-11-05,2022-11-05'

In [19]:
ACCESS_KEY = '243247ebaf607124b6b062d06e319452'

conn = http.client.HTTPConnection('api.mediastack.com')

params = urllib.parse.urlencode({
    'access_key': ACCESS_KEY,
    'languages' : 'en',
    'limit' : '100',
    'date' : parsed_date,
    'categories' : category.value,
    'keywords' : fill.value,
    'sources' : source.value,
    'sort' : 'published_desc'
    })

conn.request('GET', '/v1/news?{}'.format(params))

res = conn.getresponse()
data = res.read()

print(data.decode('utf-8'))

{"pagination":{"limit":100,"offset":0,"count":100,"total":4467},"data":[{"author":null,"title":"Garda\u00ed appeal for help in finding two missing teenagers travelling together","description":"Kelsey Kenny and Seamus Hynes were last see yesterday afternoon in the Cloghernagh area of Co Monaghan.","url":"https:\/\/www.thejournal.ie\/missing-teenagers-2-5911518-Nov2022\/","source":"thejournal","image":"https:\/\/c1.thejournal.ie\/media\/2022\/11\/blank-2-grids-collage-230x150.png","category":"general","language":"en","country":"ie","published_at":"2022-11-05T08:51:17+00:00"},{"author":"Source: Cyprus News Agency","title":"Anastasiades to meet Macron, Netanyahu","description":"President Nicos Anastasiades\u0026#8217; will meet his French counterpart Emmanuel Macron next week as he makes several trips abroad including to Egypt, Israel and France. According to the director of the president\u0026#8217;s press office, Andreas Iosif, on Sunday, Anastasiades will depart for Egypt to participate

In [20]:
dict_data = json.loads(data.decode('utf-8'))

In [21]:
if(dict_data['pagination']['count'] != 0):
    df = pd.DataFrame()
    for x in dict_data['data']:
        df = df.append(x, ignore_index=True)
    
elif (dict_data['pagination']['count'] == 0):
    print("Cancel")

In [22]:
title_df = df['title']

In [23]:
title_df

0     Gardaí appeal for help in finding two missing ...
1                Anastasiades to meet Macron, Netanyahu
2     TTEC Holdings, Inc. (NASDAQ:TTEC) Position Boo...
3     Victory Capital Management Inc. Sells 11,347 S...
4     Victory Capital Management Inc. Has $1.04 Mill...
                            ...                        
95    Charter Communications (NASDAQ:CHTR) Price Tar...
96    SVB Leerink Increases Gilead Sciences (NASDAQ:...
97    CareTrust REIT (NASDAQ:CTRE) PT Lowered to $22.00
98    U.S. Silica (NYSE:SLCA) Price Target Increased...
99    China Renaissance Cuts SEA (NYSE:SE) Price Tar...
Name: title, Length: 100, dtype: object